In [1]:
import torch 
import torch.nn as nn
#import torchvision
#import torchvision.transforms as transforms
from dataset import Dataset
from ConvNet import ConvNet
import numpy as np
import lib
from sklearn import metrics as sk_metrics
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#%load_ext autoreload
#%autoreload 2

In [2]:
# Datasets

partition = np.load('ID.npy')# IDs
angry_labels = np.load('labels_angry.npy')# Labels
fear_labels = np.load('labels_fear.npy')# Labels
happy_labels = np.load('labels_happy.npy')# Labels
sad_labels = np.load('labels_sad.npy')# Labels

partition = partition.item()

angry_labels = angry_labels.item() 
fear_labels = fear_labels.item()
happy_labels = happy_labels.item()
sad_labels = sad_labels.item()

# Drop irrelevant classes and reorder
# anger, anticipation, disgust, fear, joy, sadness
#  0,     #1,         ,   2     , 7,   8, 15
#for i in range(labels.shape[0]): # pick an irrelevant column as dummy column
#    if labels[i, 1] == 0. & :
#        labels[i,2] = 1
#    else: labels[i,2] = 0
#labels = labels[:,[0,1,2,7,8,15]]  # column 2 is useless, set to 0 in the next line
#labels[:,2] = 0. 



def init_weights(m):
    if type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
        #m.bias.data.fill_(0.01)



In [18]:
# train angry

# Hyper parameters
num_epochs = 5
num_classes = 1
learning_rate = 0.001
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 4}
# Generators
angry_training_set = Dataset(partition['train'], angry_labels)
angry_training_generator = torch.utils.data.DataLoader(angry_training_set, **params)

angry_test_set = Dataset(partition['test'], angry_labels)
angry_test_generator = torch.utils.data.DataLoader(angry_test_set, **params)


angry_model = ConvNet(num_classes).to(device)
angry_model.apply(init_weights)
angry_model.train()

# Loss and optimizer
#criterion = nn.MultiLabelSoftMarginLoss()
#criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss(pos_weight = torch.FloatTensor([7.333]))
#criterion = nn.MultiLabelMarginLoss()
optimizer = torch.optim.Adam(angry_model.parameters(), lr=learning_rate)

# Train the model
total_step = len(angry_training_generator)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(angry_training_generator):
        # print(images)
        
        images = images.to(device).float()
        labels = labels.to(device).float()
        # print (images.shape)
        # Forward pass
        outputs = angry_model(images)
        #print (labels)
        #print(outputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ( (i+1) % 4) == 0:
            print(labels)
            print(outputs)
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

torch.save(angry_model.state_dict(), 'model/angry_model.pth')

Epoch [1/5], Step [1/8], Loss: 1.0689
Epoch [1/5], Step [2/8], Loss: 3.8677
Epoch [1/5], Step [3/8], Loss: 1.9292
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
    

Epoch [4/5], Step [5/8], Loss: 0.8867
Epoch [4/5], Step [6/8], Loss: 0.8608
Epoch [4/5], Step [7/8], Loss: 1.4801
tensor([[0.],
        [0.]])
tensor([[-18.8385],
        [ -8.7608]], grad_fn=<ThAddmmBackward>)
Epoch [4/5], Step [8/8], Loss: 0.0001
Epoch [5/5], Step [1/8], Loss: 0.9690
Epoch [5/5], Step [2/8], Loss: 0.9969
Epoch [5/5], Step [3/8], Loss: 0.5698
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
       

In [29]:
# Test angry model
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
angry_model.eval()
sg = nn.Sigmoid()
with torch.no_grad():
    correct = 0
    total = 0
    label_sum = 0
    for images, labels in angry_test_generator:
        images = images.to(device).float()
        labels = labels.to(device).float()
        outputs = angry_model(images)
        #_, predicted = torch.max(outputs.data, 1)
        predicted =sg(outputs) >= 0.6
        #print ('predicted:', predicted.float())
        #print ('target: ', labels)
        #average_precision = sk_metrics.average_precision_score(labels, predicted)
        total += labels.size(0)
        correct += (predicted.float() == labels).sum().item()
        label_sum += labels.sum().item()
    print('Test accuracy: {} '.format(correct/total))
    print('class 1: {}, class 0: {} '.format(label_sum/total, 1 - label_sum/total))


Test accuracy: 0.78 
class 1: 0.12, class 0: 0.88 


In [ ]:
##### train fear #####

In [13]:
# train fear

# Hyper parameters
num_epochs = 5
num_classes = 1
learning_rate = 0.001
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 4}
# Generators
fear_training_set = Dataset(partition['train'], fear_labels)
fear_training_generator = torch.utils.data.DataLoader(fear_training_set, **params)

fear_test_set = Dataset(partition['test'], fear_labels)
fear_test_generator = torch.utils.data.DataLoader(fear_test_set, **params)

num_classes = 1

fear_model = ConvNet(num_classes).to(device)
fear_model.apply(init_weights)
fear_model.train()

# Loss and optimizer
#criterion = nn.MultiLabelSoftMarginLoss()
#criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss(pos_weight = torch.FloatTensor([1.1834]))
#criterion = nn.MultiLabelMarginLoss()
optimizer = torch.optim.Adam(fear_model.parameters(), lr=learning_rate)

# Train the model
total_step = len(fear_training_generator)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(fear_training_generator):
        # print(images)
        
        images = images.to(device).float()
        labels = labels.to(device).float()
        # print (images.shape)
        # Forward pass
        outputs = fear_model(images)
        #print (labels)
        #print(outputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ( (i+1) % 4) == 0:
            print(labels)
            print(outputs)
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

torch.save(fear_model.state_dict(), 'model/fear_model.pth')


Epoch [1/5], Step [1/8], Loss: 0.7327
Epoch [1/5], Step [2/8], Loss: 5.6996
Epoch [1/5], Step [3/8], Loss: 0.7637
tensor([[0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
    

Epoch [4/5], Step [5/8], Loss: 0.4618
Epoch [4/5], Step [6/8], Loss: 0.4377
Epoch [4/5], Step [7/8], Loss: 0.3906
tensor([[0.],
        [0.]])
tensor([[ 0.0466],
        [-1.2573]], grad_fn=<ThAddmmBackward>)
Epoch [4/5], Step [8/8], Loss: 0.4835
Epoch [5/5], Step [1/8], Loss: 0.7032
Epoch [5/5], Step [2/8], Loss: 0.4739
Epoch [5/5], Step [3/8], Loss: 0.3971
tensor([[0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [

In [14]:
torch.save(fear_model.state_dict(), 'model/fear_model_1pass.pth')

In [15]:
# Test fear model
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
fear_model.eval()
sg = nn.Sigmoid()
with torch.no_grad():
    correct = 0
    total = 0
    label_sum = 0
    for images, labels in fear_test_generator:
        images = images.to(device).float()
        labels = labels.to(device).float()
        outputs = fear_model(images)
        #_, predicted = torch.max(outputs.data, 1)
        predicted = sg(outputs) >= 0.5
        #print ('predicted:', predicted.float())
        #print ('target: ', labels)
        #average_precision = sk_metrics.average_precision_score(labels, predicted)
        total += labels.size(0)
        correct += (predicted.float() == labels).sum().item()
        label_sum += labels.sum().item()
    print('Test accuracy: {} '.format(correct/total))
    print('class 1: {}, class 0: {} '.format(label_sum/total, 1 - label_sum/total))



Test accuracy: 0.6 
class 1: 0.44, class 0: 0.56 


In [59]:
##### train happy #####

In [8]:
# train happy

# Hyper parameters
num_epochs = 10
num_classes = 1
learning_rate = 0.001
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 4}
# Generators
happy_training_set = Dataset(partition['train'], happy_labels)
happy_training_generator = torch.utils.data.DataLoader(happy_training_set, **params)

happy_test_set = Dataset(partition['test'], happy_labels)
happy_test_generator = torch.utils.data.DataLoader(happy_test_set, **params)

num_classes = 1

happy_model = ConvNet(num_classes).to(device)
happy_model.apply(init_weights)
happy_model.train()

# Loss and optimizer
#criterion = nn.MultiLabelSoftMarginLoss()
#criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss(pos_weight = torch.FloatTensor([1.7322]))
#criterion = nn.MultiLabelMarginLoss()
optimizer = torch.optim.Adam(happy_model.parameters(), lr=learning_rate)

# Train the model
total_step = len(happy_training_generator)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(happy_training_generator):
        # print(images)
        
        images = images.to(device).float()
        labels = labels.to(device).float()
        # print (images.shape)
        # Forward pass
        outputs = happy_model(images)
        #print (labels)
        #print(outputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ( (i+1) % 4) == 0:
            print(labels)
            print(outputs)
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

torch.save(happy_model.state_dict(), 'model/happy_model.pth')



Epoch [1/10], Step [1/8], Loss: 0.8639
Epoch [1/10], Step [2/8], Loss: 4.2652
Epoch [1/10], Step [3/8], Loss: 12.3078
tensor([[1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],


Epoch [4/10], Step [5/8], Loss: 1.9790
Epoch [4/10], Step [6/8], Loss: 1.9801
Epoch [4/10], Step [7/8], Loss: 1.6074
tensor([[0.],
        [1.]])
tensor([[-1.3038],
        [10.3707]], grad_fn=<ThAddmmBackward>)
Epoch [4/10], Step [8/8], Loss: 0.1201
Epoch [5/10], Step [1/8], Loss: 0.4944
Epoch [5/10], Step [2/8], Loss: 0.9091
Epoch [5/10], Step [3/8], Loss: 0.7639
tensor([[1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
  

Epoch [8/10], Step [3/8], Loss: 0.2209
tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.]])
tensor([[ -4.8847],
        [ -3.0821],
        [ -1.6198],
    

In [9]:
#torch.save(happy_model.state_dict(), 'model/happy_model_1pass.pth')



In [10]:
# Test happy model
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
happy_model.eval()
sg = nn.Sigmoid()
with torch.no_grad():
    correct = 0
    total = 0
    label_sum = 0
    for images, labels in happy_test_generator:
        images = images.to(device).float()
        labels = labels.to(device).float()
        outputs = happy_model(images)
        #_, predicted = torch.max(outputs.data, 1)
        predicted = sg(outputs) >= 0.5
        #print ('predicted:', predicted.float())
        #print ('target: ', labels)
        #average_precision = sk_metrics.average_precision_score(labels, predicted)
        total += labels.size(0)
        correct += (predicted.float() == labels).sum().item()
        label_sum += labels.sum().item()
    print('Test accuracy: {} '.format(correct/total))
    print('class 1: {}, class 0: {} '.format(label_sum/total, 1 - label_sum/total))




Test accuracy: 0.7 
class 1: 0.32, class 0: 0.6799999999999999 


In [ ]:
### sad ###

In [3]:
# train sad

# Hyper parameters
num_epochs = 5
num_classes = 1
learning_rate = 0.0001
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 4}
# Generators
sad_training_set = Dataset(partition['train'], sad_labels)
sad_training_generator = torch.utils.data.DataLoader(sad_training_set, **params)

sad_test_set = Dataset(partition['test'], sad_labels)
sad_test_generator = torch.utils.data.DataLoader(sad_test_set, **params)

num_classes = 1

sad_model = ConvNet(num_classes).to(device)
sad_model.apply(init_weights)
sad_model.train()

# Loss and optimizer
#criterion = nn.MultiLabelSoftMarginLoss()
#criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss(pos_weight = torch.FloatTensor([14.136]))
#criterion = nn.MultiLabelMarginLoss()
optimizer = torch.optim.Adam(sad_model.parameters(), lr=learning_rate)

# Train the model
total_step = len(sad_training_generator)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(sad_training_generator):
        # print(images)
        
        images = images.to(device).float()
        labels = labels.to(device).float()
        # print (images.shape)
        # Forward pass
        outputs = sad_model(images)
        #print (labels)
        #print(outputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ( (i+1) % 4) == 0:
            print(labels)
            print(outputs)
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

torch.save(sad_model.state_dict(), 'model/sad_model_logit.pth')




Epoch [1/5], Step [1/8], Loss: 1.3739
Epoch [1/5], Step [2/8], Loss: 0.9320
Epoch [1/5], Step [3/8], Loss: 1.0974
tensor([[0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
    

Epoch [4/5], Step [5/8], Loss: 0.7763
Epoch [4/5], Step [6/8], Loss: 0.7595
Epoch [4/5], Step [7/8], Loss: 0.8165
tensor([[0.],
        [0.]])
tensor([[-2.2206],
        [ 0.1441]], grad_fn=<ThAddmmBackward>)
Epoch [4/5], Step [8/8], Loss: 0.4354
Epoch [5/5], Step [1/8], Loss: 0.9560
Epoch [5/5], Step [2/8], Loss: 0.5482
Epoch [5/5], Step [3/8], Loss: 0.7119
tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [

In [12]:
# Test sad model
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
sad_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    label_sum = 0
    for images, labels in sad_test_generator:
        images = images.to(device).float()
        labels = labels.to(device).float()
        outputs = sad_model(images)
        #_, predicted = torch.max(outputs.data, 1)
        sg = nn.Sigmoid()
        predicted = sg(outputs) >= 0.4
        print ('predicted:', predicted.float())
        print ('target: ', labels)
        #average_precision = sk_metrics.average_precision_score(labels, predicted)
        total += labels.size(0)
        correct += (predicted.float() == labels).sum().item()
        label_sum += labels.sum().item()
    print('Test accuracy: {} '.format(correct/total))
    print('class 1: {}, class 0: {} '.format(label_sum/total, 1 - label_sum/total))




predicted: tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])
target:  tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],